**Universidade Federal do Amazonas**\
**Instituto de Computação - Icomp**\
**Disciplina: Inteligência Artificial - Período: 2024/2**\
**Trabalho Prático III**

**Equipe:**
* Caio Antunes (@icomp.ufam.edu.br)
* Felipe Spitale (felipe.spitale@icomp.ufam.edu.br)
* Marcello Cipriano (marcello.cipriano@icomp.ufam.edu.br)
* Pedro Henrique (@icomp.ufam.edu.br)

**Escopo:**

É dada uma instância de 100 trens de Michalsky, gerada aleatoriamente, contendo diversos atributos como: quantidade de carros (locomotiva e vagões); formato e comprimento dos vagões; formato e quantidade de cargas; a quantidade de eixo com rodas; etc. Os dados foram apresentados em um arquivo csv de nome: ***trains-uptated.csv***.

Propõe-se que os trens teriam seus destinos considerados para EAST ou WEST conforme sua apresentação na lista, convencionando que os trens da esquerda (1-25 e 51-75) iriam para EAST e os da direita (26-50 e 76-100) iriam para WEST.

A ideia do estudo seria, primeiro, identificar, por análise, as características que poderiam evidenciar padrões de determinação da direção dos trens, usando para tanto algoritmos de ***clustering*** e um programa auxiliar fornecido capaz de apontar similaridades entre atributos, o ***split_dataset.py***. Dessa análise, o objetivo era extrair ou gerar supostos ***axiomas*** que serviriam de base para a criação de um modelo LTN que possa testar as regras para classificação dos trens no dataset, com a implementação de uma solucão em LTNTorch.





### **Questão 1:**

**Parte A)**\
**Agrupar trens por similaridade usando algoritmo de clustering:**

Carregamento das DEPENDÊNCIAS:

In [ ]:
pip install keras-rectified-adam keras.utils tqdm numpy pandas scikit-learn tensorflow matplotlib pandas_ods_reader

In [ ]:
#Basic imports
from pandas_ods_reader import read_ods
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

#sklearn imports
from sklearn.decomposition import PCA #Principal Component Analysis
from sklearn.manifold import TSNE     #T-Distributed Stochastic Neighbor Embedding
from sklearn.cluster import KMeans    #K-Means Clustering
from sklearn.preprocessing import StandardScaler #used for 'Feature Scaling'
from sklearn.preprocessing import LabelEncoder


#plotly imports
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

Pré-processamento do dataset para preparar aplicação do clustering

In [ ]:
trains100 = '/content/trains-uptated.csv'
trains = '/content/trains-data_coded.csv'

In [ ]:
str_att = {
     "length": ["short", "long"],
     "shape": [
          'closedblopnrect',
          'closedrect',
          'closedtrap',
          'closedushaped',
          'dblopnrect',
          'ellipse',
          'hexagon',
          'jaggedrect',
          'openrect',
          'opentrap',
          'slopetopdblopnrect',
          'slopetoprect',
          'slopetoptrap',
          'slopetopushaped',
          'ushaped',
      ],
     "load_shape": [
          'circlelod',
          'hexagonlod',
          'rectanglod',
          'trianglod',
     ],

     "Class_attribute": ["west", "east"],
 }

def read_data(path):
    # Lê o CSV apenas uma vez
    df = pd.read_csv(path, delimiter=",", keep_default_na=False)

    # Processa as colunas com base em str_att
    for k in df.columns:
        for att in str_att:
            if k.startswith(att):  # Verifica se o nome da coluna começa com uma chave de str_att
                for i, val in enumerate(df[k]):
                    if val in str_att[att]:
                        # Exceção para "west" e "east" (Class_attribute)
                        if att == "Class_attribute":
                            if val == "west":
                                df.at[i, k] = 0
                            elif val == "east":
                                df.at[i, k] = 1
                        else:
                            df.at[i, k] = str_att[att].index(val) + 1

    # Substituições globais no DataFrame
    df.replace("\0", 0, inplace=True)
    df.replace("None", -1, inplace=True)
    df.replace("none", -1, inplace=True)

    df.index = df.index + 1

    return df

data = read_data(trains100)
data


<ipython-input-48-eb99ec132a4d>:51: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



,Number_of_cars,Number_of_different_loads,num_wheels1,length1,shape1,num_loads1,load_shape1,num_wheels2,length2,shape2,num_loads2,load_shape2,num_wheels3,length3,shape3,num_loads3,load_shape3,num_wheels4,length4,shape4,num_loads4,load_shape4,Rectangle_next_to_rectangle,Rectangle_next_to_triangle,Rectangle_next_to_hexagon,Rectangle_next_to_circle,Triangle_next_to_triangle,Triangle_next_to_hexagon,Triangle_next_to_circle,Hexagon_next_to_hexagon,Hexagon_next_to_circle,Circle_next_to_circle,Class_attribute
1,4,2,2,2,9,1,3,2,1,9,1,4,2,2,2,3,3,-1,-1,-1,-1,-1,0,1,0,0,0,0,0,0,0,0,1
2,4,2,2,1,9,2,1,2,1,3,1,3,2,1,9,1,1,-1,-1,-1,-1,-1,0,0,0,1,0,0,0,0,0,0,1
3,4,3,2,1,15,1,4,2,1,9,1,3,2,1,2,1,1,-1,-1,-1,-1,-1,0,1,0,1,0,0,0,0,0,0,1
4,5,3,2,1,10,1,1,2,1,15,1,4,2,2,9,3,3,2,1,9,1,3,1,1,0,0,0,0,1,0,0,0,1
5,4,3,2,1,2,1,4,2,1,3,1,1,2,1,15,1,3,-1,-1,-1,-1,-1,0,0,0,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,5,3,2,1,9,3,3,2,1,7,1,1,2,1,10,1,4,2,1,5,1,4,0,0,0,1,1,0,1,0,0,0,0
97,3,2,2,1,7,2,1,2,2,2,3,3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,1,0,0,0,0,0,0,0
98,3,2,2,1,12,1,1,3,2,2,1,3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,1,0,0,0,0,0,0
99,3,1,2,1,12,1,4,2,1,9,1,4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,1,0,0,0,0,0,0


In [ ]:
# Normalização dos dados
scaler = StandardScaler()
data_copy = data.copy()
data_scaled = pd.DataFrame(scaler.fit_transform(data_copy), columns=data_copy.columns)

print(data_scaled.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Number_of_cars               100 non-null    float64
 1   Number_of_different_loads    100 non-null    float64
 2   num_wheels1                  100 non-null    float64
 3   length1                      100 non-null    float64
 4   shape1                       100 non-null    float64
 5   num_loads1                   100 non-null    float64
 6   load_shape1                  100 non-null    float64
 7   num_wheels2                  100 non-null    float64
 8   length2                      100 non-null    float64
 9   shape2                       100 non-null    float64
 10  num_loads2                   100 non-null    float64
 11  load_shape2                  100 non-null    float64
 12  num_wheels3                  100 non-null    float64
 13  length3              

In [ ]:
# Aplicação de Clustering (K-Means)
kmeans = KMeans(n_clusters=2,random_state=43)
kmeans.fit(data_scaled)
data['Cluster'] = kmeans.predict(data_scaled)


In [ ]:
# Redução de Dimensionalidade (PCA e T-SNE)
pca_2d = PCA(n_components=2)
PCs_2d = pca_2d.fit_transform(data_scaled)
data['PC1_2d'] = PCs_2d[:, 0]
data['PC2_2d'] = PCs_2d[:, 1]


In [ ]:
from sklearn.metrics import silhouette_score

silhouette = silhouette_score(data_scaled, kmeans.labels_)
print(f"Silhouette Score: {silhouette}")


Silhouette Score: 0.19968777437095334


### Os Trens são visualizados de forma interativa no espaço 2D usando Plotly, cada ponto representa um trem. ###

In [ ]:
trace1 = go.Scatter(
    x = data[data['Cluster'] == 0]['PC1_2d'],
    y = data[data['Cluster'] == 0]['PC2_2d'],
    mode = "markers+text",
    name = "Cluster 0",
    marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
    text = data[data['Cluster'] == 0].index,
    textposition = "top center",
    textfont = dict(size = 10)
)

trace2 = go.Scatter(
    x = data[data['Cluster'] == 1]['PC1_2d'],
    y = data[data['Cluster'] == 1]['PC2_2d'],
    mode = "markers+text",
    name = "Cluster 1",
    marker = dict(color = 'rgba(255, 128, 2, 0.8)'),
    text = data[data['Cluster'] == 1].index,
    textposition = "top center",
    textfont = dict(size = 10)
)


layout = dict(title = "Visualização dos Clusters (PCA 2D)",
              xaxis= dict(title= 'PC1', ticklen= 5, zeroline= False),
              yaxis= dict(title= 'PC2', ticklen= 5, zeroline= False)
             )

fig = dict(data = [trace1, trace2], layout = layout)
iplot(fig)


In [ ]:
data_scaled

,Number_of_cars,Number_of_different_loads,num_wheels1,length1,shape1,num_loads1,load_shape1,num_wheels2,length2,shape2,num_loads2,load_shape2,num_wheels3,length3,shape3,num_loads3,load_shape3,num_wheels4,length4,shape4,num_loads4,load_shape4,Rectangle_next_to_rectangle,Rectangle_next_to_triangle,Rectangle_next_to_hexagon,Rectangle_next_to_circle,Triangle_next_to_triangle,Triangle_next_to_hexagon,Triangle_next_to_circle,Hexagon_next_to_hexagon,Hexagon_next_to_circle,Circle_next_to_circle,Class_attribute,Cluster,PC1_2d,PC2_2d
0,-0.151717,-0.277350,-0.229416,2.708013,0.141401,-0.208664,0.427164,-0.436436,-0.608164,0.260243,-0.242759,1.023101,0.454257,1.088410,-0.592126,1.884233,0.778681,-0.774597,-0.758621,-0.670599,-0.743358,-0.718954,-0.420084,1.333333,-0.229416,-0.766356,-0.468521,-0.252646,-0.733799,0.0,-0.100504,-0.436436,1.0,0.782881,-0.056349,-0.257743
1,-0.151717,-0.277350,-0.229416,-0.369274,0.141401,1.688285,-1.045815,-0.436436,-0.608164,-1.555406,-0.242759,0.276312,0.454257,0.203524,0.771322,0.245770,-0.295362,-0.774597,-0.758621,-0.670599,-0.743358,-0.718954,-0.420084,-0.750000,-0.229416,1.304877,-0.468521,-0.252646,-0.733799,0.0,-0.100504,-0.436436,1.0,0.782881,-0.450600,-0.889336
2,-0.151717,1.109400,-0.229416,-0.369274,1.742164,-0.208664,1.163654,-0.436436,-0.608164,0.260243,-0.242759,0.276312,0.454257,0.203524,-0.592126,0.245770,-0.295362,-0.774597,-0.758621,-0.670599,-0.743358,-0.718954,-0.420084,1.333333,-0.229416,1.304877,-0.468521,-0.252646,-0.733799,0.0,-0.100504,-0.436436,1.0,0.782881,-0.218748,0.259212
3,1.112588,1.109400,-0.229416,-0.369274,0.408195,-0.208664,-1.045815,-0.436436,-0.608164,2.075892,-0.242759,1.023101,0.454257,1.088410,0.771322,1.884233,0.778681,1.161895,0.965517,1.273165,1.005720,1.349978,2.380476,1.333333,-0.229416,-0.766356,-0.468521,-0.252646,1.362770,0.0,-0.100504,-0.436436,1.0,-1.277333,1.501036,0.640072
4,-0.151717,1.109400,-0.229416,-0.369274,-1.726156,-0.208664,1.163654,-0.436436,-0.608164,-1.555406,-0.242759,-1.217266,0.454257,0.203524,1.939991,0.245770,0.778681,-0.774597,-0.758621,-0.670599,-0.743358,-0.718954,-0.420084,-0.750000,-0.229416,1.304877,-0.468521,-0.252646,1.362770,0.0,-0.100504,-0.436436,1.0,0.782881,-0.103588,-1.600239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.112588,1.109400,-0.229416,-0.369274,0.141401,3.585234,0.427164,-0.436436,-0.608164,-0.344973,-0.242759,-1.217266,0.454257,0.203524,0.966100,0.245770,1.315703,1.161895,0.965517,0.495660,1.005720,1.867211,-0.420084,-0.750000,-0.229416,1.304877,2.134375,-0.252646,1.362770,0.0,-0.100504,-0.436436,-1.0,-1.277333,1.070434,0.045127
96,-1.416021,-0.277350,-0.229416,-0.369274,-0.392187,1.688285,-1.045815,-0.436436,1.644294,-1.858014,2.791727,0.276312,-1.642313,-1.566249,-1.176461,-1.392694,-1.369405,-0.774597,-0.758621,-0.670599,-0.743358,-0.718954,-0.420084,-0.750000,-0.229416,1.304877,-0.468521,-0.252646,-0.733799,0.0,-0.100504,-0.436436,-1.0,0.782881,-1.319731,0.599642
97,-1.416021,-0.277350,-0.229416,-0.369274,0.941782,-0.208664,-1.045815,2.291288,1.644294,-1.858014,-0.242759,0.276312,-1.642313,-1.566249,-1.176461,-1.392694,-1.369405,-0.774597,-0.758621,-0.670599,-0.743358,-0.718954,-0.420084,-0.750000,-0.229416,-0.766356,2.134375,-0.252646,-0.733799,0.0,-0.100504,-0.436436,-1.0,0.782881,-1.388388,0.922018
98,-1.416021,-1.664101,-0.229416,-0.369274,0.941782,-0.208664,1.163654,-0.436436,-0.608164,0.260243,-0.242759,1.023101,-1.642313,-1.566249,-1.176461,-1.392694,-1.369405,-0.774597,-0.758621,-0.670599,-0.743358,-0.718954,-0.420084,-0.750000,-0.229416,-0.766356,2.134375,-0.252646,-0.733799,0.0,-0.100504,-0.436436,-1.0,0.782881,-1.277567,1.658485


**Parte B)**\
**Buscar similaridades por análise e gerar supostos axiomas:**

In [ ]:
data

,Number_of_cars,Number_of_different_loads,num_wheels1,length1,shape1,num_loads1,load_shape1,num_wheels2,length2,shape2,num_loads2,load_shape2,num_wheels3,length3,shape3,num_loads3,load_shape3,num_wheels4,length4,shape4,num_loads4,load_shape4,Rectangle_next_to_rectangle,Rectangle_next_to_triangle,Rectangle_next_to_hexagon,Rectangle_next_to_circle,Triangle_next_to_triangle,Triangle_next_to_hexagon,Triangle_next_to_circle,Hexagon_next_to_hexagon,Hexagon_next_to_circle,Circle_next_to_circle,Class_attribute,Cluster,PC1_2d,PC2_2d
1,4,2,2,2,9,1,3,2,1,9,1,4,2,2,2,3,3,-1,-1,-1,-1,-1,0,1,0,0,0,0,0,0,0,0,1,1,-0.160943,-0.418791
2,4,2,2,1,9,2,1,2,1,3,1,3,2,1,9,1,1,-1,-1,-1,-1,-1,0,0,0,1,0,0,0,0,0,0,1,1,-1.286986,-1.445032
3,4,3,2,1,15,1,4,2,1,9,1,3,2,1,2,1,1,-1,-1,-1,-1,-1,0,1,0,1,0,0,0,0,0,0,1,1,-0.624779,0.421179
4,5,3,2,1,10,1,1,2,1,15,1,4,2,2,9,3,3,2,1,9,1,3,1,1,0,0,0,0,1,0,0,0,1,0,4.287202,1.040017
5,4,3,2,1,2,1,4,2,1,3,1,1,2,1,15,1,3,-1,-1,-1,-1,-1,0,0,0,1,0,0,1,0,0,0,1,1,-0.295865,-2.600139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,5,3,2,1,9,3,3,2,1,7,1,1,2,1,10,1,4,2,1,5,1,4,0,0,0,1,1,0,1,0,0,0,0,0,3.057333,0.073324
97,3,2,2,1,7,2,1,2,2,2,3,3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,1,0,0,0,0,0,0,0,1,-3.769367,0.974325
98,3,2,2,1,12,1,1,3,2,2,1,3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,1,0,0,0,0,0,0,1,-3.965461,1.498136
99,3,1,2,1,12,1,4,2,1,9,1,4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,1,0,0,0,0,0,0,1,-3.648938,2.694780


Separação dos clusters em dois dataframes para auxiliar análise de características e extração dos axiomas.

In [ ]:
#Cluster 0
data_0 = data[data['Cluster'] == 0]

#Cluster 1
data_1 = data[data['Cluster'] == 1]

cluster_0 = [(idx, data.loc[idx, 'Class_attribute']) for idx in data[data['Cluster'] == 0].index]
cluster_1 = [(idx, data.loc[idx, 'Class_attribute']) for idx in data[data['Cluster'] == 1].index]

print("Cluster 0:", cluster_0)
print("Cluster 1:", cluster_1)


Cluster 0: [(4, 1), (7, 1), (9, 1), (11, 1), (12, 1), (14, 1), (17, 1), (18, 1), (24, 1), (25, 1), (26, 0), (30, 0), (31, 0), (33, 0), (34, 0), (36, 0), (44, 0), (46, 0), (50, 0), (52, 1), (56, 1), (59, 1), (60, 1), (61, 1), (63, 1), (66, 1), (68, 1), (71, 1), (74, 1), (78, 0), (79, 0), (80, 0), (82, 0), (85, 0), (87, 0), (93, 0), (94, 0), (96, 0)]
Cluster 1: [(1, 1), (2, 1), (3, 1), (5, 1), (6, 1), (8, 1), (10, 1), (13, 1), (15, 1), (16, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (27, 0), (28, 0), (29, 0), (32, 0), (35, 0), (37, 0), (38, 0), (39, 0), (40, 0), (41, 0), (42, 0), (43, 0), (45, 0), (47, 0), (48, 0), (49, 0), (51, 1), (53, 1), (54, 1), (55, 1), (57, 1), (58, 1), (62, 1), (64, 1), (65, 1), (67, 1), (69, 1), (70, 1), (72, 1), (73, 1), (75, 1), (76, 0), (77, 0), (81, 0), (83, 0), (84, 0), (86, 0), (88, 0), (89, 0), (90, 0), (91, 0), (92, 0), (95, 0), (97, 0), (98, 0), (99, 0), (100, 0)]


In [ ]:
# Para Cluster 0
count_1_cluster_0 = sum(1 for _, value in cluster_0 if value == 1)
count_0_cluster_0 = sum(1 for _, value in cluster_0 if value == 0)

# Para Cluster 1
count_1_cluster_1 = sum(1 for _, value in cluster_1 if value == 1)
count_0_cluster_1 = sum(1 for _, value in cluster_1 if value == 0)

# Exibindo os resultados
print(f"Cluster 0 - Quantidade de trens para EAST: {count_1_cluster_0}, e para WEST: {count_0_cluster_0}")
print(f"Cluster 1 - Quantidade de trens para EAST: {count_1_cluster_1}, e para WEST: {count_0_cluster_1}")

Cluster 0 - Quantidade de trens para EAST: 20, e para WEST: 18
Cluster 1 - Quantidade de trens para EAST: 30, e para WEST: 32


Com a aplicação do algoritmo de clustering, foi possível conferir nitidamente a formação de um agrupamento (Cluster 0) contendo apenas os trens com 5 carros (1 locomotiva e 4 vagões) e um outro agrupamento (Cluster 1) contendo os demais exemplos de trens com menos de 5 carros.

Ocorre que essa divisão ainda mantém muita aproximada de 50% a proporção de trens que se destinam a cada lado (EAST e WEST) em ambos os clusters, de modo que exige a adoção de outros critérios para análise e identificação de potenciais regras de determinação do direicionamento aplicáveis a essa instância de Michalsky.

Para essa tarefa complementar, definimos abaixo então a função ***split_dataset***, a partir do código em python fornecido na especificação do trabalho, com algumas adaptações para extrair dados relevantes para a síntese dos axiomas com base na análise de padrões dos clusters, assim como da totalidade da amostra.

In [ ]:
def split_dataset(dataframe,coluna):

  distinct_values = dataframe[coluna].unique()
  #list_df = []

  for value in distinct_values:
      indices = dataframe[dataframe[coluna] == value].index.tolist()
      index_value_tuples = [(idx, dataframe.at[idx, 'Class_attribute']) for idx in indices]
      count_1_cluster = sum(1 for _, value in index_value_tuples if value == 1)
      count_0_cluster = sum(1 for _, value in index_value_tuples if value == 0)

      # Exibindo os resultados
      print(f"Valor: {value}")
      print(f"Índices e Direção (1-EAST, 0-WEST): {index_value_tuples}")
      print(f"Quantidade de índices: {len(indices)}")
      print(f"Quantidade de EAST: {count_1_cluster}, WEST: {count_0_cluster}")
      print("-" * 30)  # Separador entre as saídas

      #print(list_df)

In [ ]:
# Aplicação da função split_dataset para cada coluna dos Clusters
for coluna in data.columns[:-4]:
  print("\nAtributo:", coluna)
  print("\nCluster 0:\n")
  split_dataset(data_0,coluna)
  print("\nCluster 1:\n")
  split_dataset(data_1,coluna)


Apenas com a separação em clusters, e na forma como acima foi descrita, não foi possível definir aspectos determinantes que possam confirmar a orientação do destino dos trens. Apenas um aspecto chama um pouco mais de atenção dentro do Cluster1 que é o fato de que a grande maioria dos trens com apenas 3 carros seguem para WEST. Todavia, percebemos a possibilidade de tornar essa suposição um pouco mais forte buscando outros fatores de similaridade aplicando a função ***split_dataset*** a todo o conjunto de trens e não exatamente separando por clusters, o que abaixo se experimentou:

In [ ]:
# Aplicar a função split_dataset para cada coluna do DataFrame todo
for coluna in data.columns[:-4]:
  print("\nAtributo:", coluna)
  split_dataset(data,coluna)

A partir da aplicação da função ***split_dataset*** a todo conjunto de dados, ficaram mais visíveis alguns pontos de similaridade que, combinados com outros já existentes, prometem definir com mais precisão alguns padrões de orientação do destino dos trens, dentre os quais identificamos:

- a quantidade de vagões longos do trem;
- um tipo de formato de carga específico (hexagonal) em qualquer vagão do trem;
- e diferentes quantidades de carga entre os vagões do trem.

Por tal motivo, resolvemos tornar expressos esses aspectos como novos atributos da tabela que define as características dos trens, o que levou a ideia de criar três novas colunas no *dataset* utilizado, criando-se a tabela completa ***data_improved***, conforme código abaixo:

In [ ]:
data_improved = pd.DataFrame(data)

value = 2

#inclui coluna que indica quantidade de vagões longos para o trem
data_improved['quant_car_long'] = data_improved[['length1','length2','length3','length4']].apply(lambda row: (row == value).sum(), axis=1)

#inclui coluna que indica se tem carga hexagonal em algum vagão
data_improved['has_load_hexagon_shape'] = (data_improved[['load_shape1','load_shape2','load_shape3','load_shape4']] == value).any(axis=1).astype(int)

#inclui coluna que indica se tem algum vagão com mais de uma quantidade da carga que leva
cols_to_convert = ['num_loads1', 'num_loads2', 'num_loads3', 'num_loads4']
data_improved[cols_to_convert] = data_improved[cols_to_convert].apply(pd.to_numeric, errors='raise')
data_improved['has_num_loads_car_above1'] = (data_improved[['num_loads1', 'num_loads2', 'num_loads3', 'num_loads4']] >= value).any(axis=1).astype(int)

colunas_relevantes = ['Number_of_cars','quant_car_long','has_load_hexagon_shape','has_num_loads_car_above1']
print(data_improved[colunas_relevantes])

Após a inclusão dessas novas colunas, aplicamos novamente a função ***split_dataset*** sobre a tabela, mas tão somente considerando a relevância das 4 colunas destacadas nos apontamentos acima:
- a coluna ***'Number_of_cars'*** percebida com aplicação do algoritmo de clustering;
- e as três novas colunas criadas: ***'quant_car_long'***; ***'has_load_hexagon_shape'***; ***'has_num_loads_car_above1'***.

In [ ]:
for coluna in colunas_relevantes:
  print("\nAtributo:", coluna)
  split_dataset(data_improved,coluna)


Dessa forma, restaram mais evidentes os pontos de similaridades investigados, tornando possível apresentar supostos axiomas para determinar a orientação dos trens com maior potencial de precisão.

Antes porém de expressar os identificados axiomas, deixemos clara a relação de predicados do problema já oferecidos na especificação do trabalho, com o acréscimo de alguns decorrentes da criação dos novos dados expressos na tabela de características encontradas:

***Predicados dados:***
1. num_cars(t, nc), em que t ∊ [1..100] e nc ∊ [3..5] (nc denota o número de carros do trem incluindo a locomotiva).
2. num_loads(t, nl) em que t ∊ [1..100] e nl ∊ [1..4].
3. num_wheels(t, c, w) em que t ∊ [1..100] e c ∊ [1..4] e w ∊ [2..3].
4. length(t, c, l) em que t ∊ [1..100] e c ∊ [1..4] e l ∊ [1..2] (1 denota curto e 2
longo)
5. shape(t, c, s) em que t ∊ [1..100] e c ∊ [1..4] e s ∊ [1..15] (um número para cada
forma).
6. num_cars_loads(t, c, ncl) em que t ∊ [1..100] e c ∊ [1..4] e ncl ∊ [0..3].
7. load_shape(t, c, ls) em que t ∊ [1..100] e c ∊ [1..4] e ls ∊ [1..4].
8. next_crc(t, c, x) em que t ∊ [1..100] e c ∊ [1..4] e x ∊ [0..1], em que o vagão c do
trem t tem um vagão adjacente com cargas em círculo.
9. next_hex(t, c, x) em que t ∊ [1..100] e c ∊ [1..4] e x ∊ [0..1], em que o vagão c
do trem t tem um vagão adjacente com cargas em hexágono.
10.next_rec(t, c, x) em que t ∊ [1..100] e c ∊ [1..4] e x ∊ [0..1], em que o vagão c
do trem t tem um vagão adjacente com cargas em retângulo.
11.next_tri(t, c, x) em que t ∊ [1..100] e c ∊ [1..4] e x ∊ [0..1], em que o vagão c do
trem t tem um vagão adjacente com cargas em triângulo.

***Predicados novos identificados:***
12. num_cars_long(t, nclg), em que t ∊ [1..100] e nclg ∊ [0..4], indicando a quantidade de vagões de comprimento longo.
13. has_num_loads_car_above1(t, nlab1), em que t ∊ [1..100] e nlab1 ∊ [0..1], indicando se o trem tem algum vagão com uma carga superior a uma unidade.

Obs.: Como predicado relativo à nova coluna *'has_load_hexagon_shape'* é possível utilizar o expresso no número 7 acima.


Conhecido os predicados aplicáveis, os **supostos axiomas** extraídos podem ser expressos conforme abaixo:

1. **num_cars_long(T,3) → west(T)** {*Explica que os trens que possuem 3 vagões longos vão para WEST*}
2. **load_shape(T,C,2) → west(T)** {*Explica que os trens que tenham algum vagão com carga hexagonal vão para WEST*}
3. **num_cars(T,4) ∧ num_cars_long(T,2) → east(T)** {*Explica que trens com 4 carros (incluido locomotiva) e 2 vagões longos vão para EAST*}
4.**num_cars(T,3) ∧ load_shape(T,1,4) ∧ load_shape(T,2,4) → west(T)** {*Explica que os trens menores (3 carros), cujos dois vagões tenham simultaneamente carga triangular, vão para WEST}
5. **num_cars(T,3) ∧ has_num_loads_car_above1(T,1) → west(T)** {*Explica que os trens menores (3 carros), que tenha pelo menos um vagão com carga em mais de uma unidade, vão para WEST


